In [ ]:
import sklearn
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plot_2d_clf_problem(X, y, h=None):
    '''
    Plots a two-dimensional labeled dataset (X,y) and, if function h(x) is given, 
    the decision surfaces.
    '''
    assert X.shape[1] == 2, "Dataset is not two-dimensional"
    if h!=None : 
        # Create a mesh to plot in
        r = 0.04  # mesh resolution
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, r),
                             np.arange(y_min, y_max, r))
        XX=np.c_[xx.ravel(), yy.ravel()]
        try:
            Z_test = h(XX)
            if Z_test.shape == ():
                # h returns a scalar when applied to a matrix; map explicitly
                Z = np.array(list(map(h,XX)))
            else :
                Z = Z_test
        except ValueError:
            # can't apply to a matrix; map explicitly
            Z = np.array(list(map(h,XX)))
        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        plt.contourf(xx, yy, Z, cmap=plt.cm.Pastel1)

    # Plot the dataset
    plt.scatter(X[:,0],X[:,1], c=y, cmap=plt.cm.tab20b, marker='o', s=50);

### 1. Logistička regresija

Ovaj zadatak bavi se probabilističkim diskriminativnim modelom, **logističkom regresijom**, koja je, unatoč nazivu, klasifikacijski model.

Logistička regresija tipičan je predstavnik tzv. **poopćenih linearnih modela** koji su oblika: $h(\mathbf{x})=f(\mathbf{w}^\intercal\tilde{\mathbf{x}})$. Logistička funkcija za funkciju $f$ koristi tzv. **logističku** (sigmoidalnu) funkciju $\sigma (x) = \frac{1}{1 + \textit{exp}(-x)}$.

### (a)  

Definirajte logističku (sigmoidalnu) funkciju $\mathrm{sigm}(x)=\frac{1}{1+\exp(-\alpha x)}$ i prikažite je za $\alpha\in\{1,2,4\}$.

In [ ]:
def sigm(x, a):
    return 1 / (1 + np.exp(-a * x))

X = np.linspace(-5, 5)

for i in [1, 2, 4]:
    y = sigm(X, i)
    plt.plot(X, y)

plt.show()

**Q**: Zašto je sigmoidalna funkcija prikladan izbor za aktivacijsku funkciju poopćenoga linearnog modela? 
</br>

**Q**: Kakav utjecaj ima faktor $\alpha$ na oblik sigmoide? Što to znači za model logističke regresije (tj. kako izlaz modela ovisi o normi vektora težina $\mathbf{w}$)?

### (b) 

Implementirajte funkciju 

> `lr_train(X, y, eta=0.01, max_iter=2000, alpha=0, epsilon=0.0001, trace=False)` 

za treniranje modela logističke regresije gradijentnim spustom (*batch* izvedba). Funkcija uzima označeni skup primjera za učenje (matrica primjera `X` i vektor oznaka `y`) te vraća $(n+1)$-dimenzijski vektor težina tipa `ndarray`. Ako je `trace=True`, funkcija dodatno vraća listu (ili matricu) vektora težina $\mathbf{w}^0,\mathbf{w}^1,\dots,\mathbf{w}^k$ generiranih kroz sve iteracije optimizacije, od 0 do $k$. Optimizaciju treba provoditi dok se ne dosegne `max_iter` iteracija, ili kada razlika u pogrešci unakrsne entropije između dviju iteracija padne ispod vrijednosti `epsilon`. Parametar `alpha` predstavlja faktor L2-regularizacije.

Preporučamo definiranje pomoćne funkcije `lr_h(x,w)` koja daje predikciju za primjer `x` uz zadane težine `w`. Također, preporučamo i funkciju `cross_entropy_error(X,y,w)` koja izračunava pogrešku unakrsne entropije modela na označenom skupu `(X,y)` uz te iste težine.

**NB:** Obratite pozornost na to da je način kako su definirane oznake ($\{+1,-1\}$ ili $\{1,0\}$) kompatibilan s izračunom funkcije gubitka u optimizacijskome algoritmu.

In [ ]:
from numpy import linalg
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(1)

def lr_h(x, w):
    x = np.append(np.ones((x.shape[0], 1)), x, axis=1)
    return 1 / (1 + np.exp(-1 * x @ w))

def cross_entropy_error(X, y, w):
    return np.mean(-y.reshape(-1, 1) * np.log(lr_h(X, w)) - (1 - y.reshape(-1, 1)) * np.log(1 - lr_h(X, w)))

def lr_train(X, y, eta=0.01, max_iter=2000, alpha=0, epsilon=0.0001, trace=False):
    
    weight_matrix = []
    last_error = 0
    w = np.zeros((X.shape[1] + 1, 1))

    for i in range(max_iter + 1):

        weight_matrix.append(w.copy())

        cur_error = cross_entropy_error(X, y, w)
        if np.abs(cur_error - last_error) < epsilon:
            break

        dw = np.zeros((X.shape[1] + 1, 1))

        for j in range(X.shape[0]):

            h = lr_h(X[j].reshape(1, -1), w)
            dw = dw - (h - y[j].reshape(1, 1)) * np.append(1, X[j]).reshape(-1, 1)

        w[0] = w[0] + eta * dw[0]
        w[1:] = w[1:] * (1 - eta * alpha) + eta * dw[1:]

    weight_matrix = np.array(weight_matrix)

    if trace:
        return w, weight_matrix
    else:
        return w


### (c)

Koristeći funkciju `lr_train`, trenirajte model logističke regresije na skupu `seven`, prikažite dobivenu granicu između klasa te  izračunajte pogrešku unakrsne entropije. 

**NB:** Pripazite da modelu date dovoljan broj iteracija.

In [ ]:
from sklearn.linear_model import LogisticRegression

seven_X = np.array([[2,1], 
                    [2,3], 
                    [1,2], 
                    [3,2], 
                    [5,2], 
                    [5,4], 
                    [6,3]])

seven_y = np.array([1, 1, 1, 1, 0, 0, 0])

w, weights_legacy = lr_train(seven_X, seven_y, trace=True, eta=0.01)

print(f'Weights:\n {w}')
print(f'Error: {cross_entropy_error(seven_X, seven_y, w)}')


model = LogisticRegression()
model.fit(seven_X, seven_y)
predict = model.predict(seven_X)
print(f'Model predict: {predict}')
print(model.coef_, model.intercept_)


plot_2d_clf_problem(seven_X, seven_y, lambda x: lr_h(x, w) >= 0.5)
plt.show()

**Q:** Koji kriterij zaustavljanja je aktiviran?

**Q:** Zašto dobivena pogreška unakrsne entropije nije jednaka nuli?

**Q:** Kako biste utvrdili da je optimizacijski postupak doista pronašao hipotezu koja minimizira pogrešku učenja? O čemu to ovisi?

**Q:** Na koji način biste preinačili kôd ako biste htjeli da se optimizacija izvodi stohastičkim gradijentnim spustom (*online learning*)?

### (d)

Prikažite na jednom grafikonu pogrešku unakrsne entropije (očekivanje logističkog gubitka) i pogrešku klasifikacije (očekivanje gubitka 0-1) na skupu `seven` kroz iteracije optimizacijskog postupka. Koristite trag težina funkcije `lr_train` iz zadatka (b) (opcija `trace=True`). Na drugom grafikonu prikažite pogrešku unakrsne entropije kao funkciju broja iteracija za različite stope učenja, $\eta\in\{0.005,0.01,0.05,0.1\}$.

In [ ]:
from sklearn.metrics import zero_one_loss

errors_entropy, errors_zero_one = [], []

for w in weights_legacy:
    errors_entropy.append(cross_entropy_error(seven_X, seven_y, w))
    errors_zero_one.append(zero_one_loss(seven_y, lr_h(seven_X, w) >= 0.5))

errors_entropy = np.array(errors_entropy)
errors_zero_one = np.array(errors_zero_one)
x_axis = np.array([x for x in range(2001)])

plt.plot(x_axis, errors_entropy)
plt.plot(x_axis, errors_zero_one)
plt.show()

num_of_iter = 2000
for lr in [0.005, 0.01, 0.05, 0.1]: #0.01, 0.05, 0.1]:
    w, weights_legacy = lr_train(seven_X, seven_y, trace=True, eta=lr, max_iter=num_of_iter)

    errors_entropy = []
    for w in weights_legacy:
        errors_entropy.append(cross_entropy_error(seven_X, seven_y, w))
    errors_entropy = np.array(errors_entropy)
    x_axis = np.array([x for x in range(num_of_iter + 1)])
    plt.plot(x_axis, errors_entropy, label=lr)

plt.legend()
plt.show()


**Q:** Zašto je pogreška unakrsne entropije veća od pogreške klasifikacije? Je li to uvijek slučaj kod logističke regresije i zašto?

**Q:** Koju stopu učenja $\eta$ biste odabrali i zašto?

### (e)

Upoznajte se s klasom [`linear_model.LogisticRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) koja implementira logističku regresiju. Usporedite rezultat modela na skupu `seven` s rezultatom koji dobivate pomoću vlastite implementacije algoritma.

**NB:** Kako ugrađena implementacija koristi naprednije verzije optimizacije funkcije, vrlo je vjerojatno da Vam se rješenja neće poklapati, ali generalne performanse modela bi trebale. Ponovno, pripazite na broj iteracija i snagu regularizacije.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(seven_X, seven_y)
predict = model.predict(seven_X)

plot_2d_clf_problem(seven_X, seven_y, model.predict)
plt.show()

### 2. Analiza logističke regresije

### (a)

Koristeći ugrađenu implementaciju logističke regresije, provjerite kako se logistička regresija nosi s vrijednostima koje odskaču. Iskoristite skup `outlier`. Prikažite granicu između klasa.

In [ ]:
outlier_X = np.append(seven_X, [[12,8]], axis=0)
outlier_y = np.append(seven_y, 0)

In [ ]:
model.fit(outlier_X, outlier_y)
predict = model.predict(outlier_X)

plot_2d_clf_problem(outlier_X, outlier_y, model.predict)
plt.show()

**Q:** Zašto se rezultat razlikuje od onog koji je dobio model klasifikacije linearnom regresijom iz prvog zadatka?

### (b)

Trenirajte model logističke regresije na skupu `seven` te na dva odvojena grafikona prikažite, kroz iteracije optimizacijskoga algoritma, (1) izlaz modela $h(\mathbf{x})$ za svih sedam primjera te (2) vrijednosti težina $w_0$, $w_1$, $w_2$.

In [ ]:
w, weights_legacy = lr_train(seven_X, seven_y, trace=True)

izlazi = [[] for x in range(7)]
tezine = [[] for x in range(3)]

for w in weights_legacy:

    output = lr_h(seven_X, w)

    for i in range(7):
        izlazi[i].append(output[i])

    for i in range(3):
        tezine[i].append(w[i])

for a in izlazi:
    a = np.array(a)

for a in tezine:
    a = np.array(a)

for i in range(7):
    plt.plot(x_axis, izlazi[i], label=f'{seven_X[i]}, {seven_y[i]}')
plt.legend()
plt.show()

for i in range(3):
    plt.plot(x_axis, tezine[i], label=f'w{i}')

plt.legend()
plt.show()

### (c)

Ponovite eksperiment iz podzadatka (b) koristeći linearno neodvojiv skup podataka `unsep`.

In [ ]:
unsep_X = np.append(seven_X, [[2,2]], axis=0)
unsep_y = np.append(seven_y, 0)

In [ ]:
w, weights_legacy = lr_train(unsep_X, unsep_y, trace=True)

plot_2d_clf_problem(unsep_X, unsep_y, lambda x: lr_h(x, w) >= 0.5)
plt.show()

izlazi = [[] for x in range(8)]
tezine = [[] for x in range(3)]

print(unsep_X.shape)

for w in weights_legacy:

    output = lr_h(unsep_X, w)

    for i in range(8):
        izlazi[i].append(output[i])

    for i in range(3):
        tezine[i].append(w[i])

for a in izlazi:
    a = np.array(a)

for a in tezine:
    a = np.array(a)

for i in range(8):
    plt.plot(x_axis, izlazi[i], label=f'{unsep_X[i]}, {unsep_y[i]}')
plt.legend()
plt.show()

for i in range(3):
    plt.plot(x_axis, tezine[i], label=f'w{i}')

plt.legend()
plt.show()

**Q:** Usporedite grafikone za slučaj linearno odvojivih i linearno neodvojivih primjera te komentirajte razliku.

### 3. Regularizirana logistička regresija

Trenirajte model logističke regresije na skupu `seven` s različitim faktorima L2-regularizacije, $\alpha\in\{0,1,10,100\}$. Prikažite na dva odvojena grafikona (1) pogrešku unakrsne entropije te (2) L2-normu vektora $\mathbf{w}$ kroz iteracije optimizacijskog algoritma.

In [ ]:
from numpy.linalg import norm

err_glob, norm_glob = [], []

for a in [0, 1, 10, 100]:

    w, weights_legacy = lr_train(seven_X, seven_y, trace=True, alpha=a)

    errors = []
    norms = []

    for w in weights_legacy:
        errors.append(cross_entropy_error(seven_X, seven_y, w))
        norms.append(norm(w, ord=2))

    errors = np.array(errors)
    norms = np.array(norms)

    err_glob.append(errors)
    norm_glob.append(norms)


for i in range(len(err_glob)):
    if i == 0:
        plt.plot(x_axis, err_glob[i], label=f'a={i}')
    else:
        plt.plot(x_axis, err_glob[i], label=f'a={10**(i - 1)}')
plt.legend()
plt.show()

for i in range(len(norm_glob)):
    if i == 0:
        plt.plot(x_axis, norm_glob[i], label=f'a={i}')
    else:
        plt.plot(x_axis, norm_glob[i], label=f'a={10**(i - 1)}')
plt.legend()
plt.show()

**Q:** Jesu li izgledi krivulja očekivani i zašto?

**Q:** Koju biste vrijednost za $\alpha$ odabrali i zašto?

### 4. Logistička regresija s funkcijom preslikavanja

Proučite funkciju [`datasets.make_classification`](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html). Generirajte i prikažite dvoklasan skup podataka s ukupno $N=100$ dvodimenzijskih ($n=2)$ primjera, i to sa dvije grupe po klasi (`n_clusters_per_class=2`). Malo je izgledno da će tako generiran skup biti linearno odvojiv, međutim to nije problem jer primjere možemo preslikati u višedimenzijski prostor značajki pomoću klase [`preprocessing.PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html), kao što smo to učinili kod linearne regresije u prvoj laboratorijskoj vježbi. Trenirajte model logističke regresije koristeći za preslikavanje u prostor značajki polinomijalnu funkciju stupnja $d=2$ i stupnja $d=3$. Prikažite dobivene granice između klasa. Možete koristiti svoju implementaciju, ali se radi brzine preporuča koristiti `linear_model.LogisticRegression`. Regularizacijski faktor odaberite po želji.

**NB:** Kao i ranije, za prikaz granice između klasa koristite funkciju `plot_2d_clf_problem`. Funkciji kao argumente predajte izvorni skup podataka, a preslikavanje u prostor značajki napravite unutar poziva funkcije `h` koja čini predikciju, na sljedeći način:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import make_classification

#plot_2d_clf_problem(X, y, lambda x : model.predict(poly.transform(x))

X, y = make_classification(n_samples=100, n_classes=2, n_clusters_per_class=2, n_redundant=0, n_informative=2, n_features=2)

plot_2d_clf_problem(X, y)
plt.show()

In [ ]:
model = LogisticRegression()

for d in [2, 3]:
    poly = PolynomialFeatures(d)
    X_extended = poly.fit_transform(X)
    model.fit(X_extended, y)
    plot_2d_clf_problem(X, y, lambda x: model.predict(poly.fit_transform(x)))
    plt.show()
    print(X_extended[0])
    print(model.coef_, model.intercept_)

In [ ]:
for d in [2, 3]:
    poly = PolynomialFeatures(d)
    X_extended = poly.fit_transform(X)
    w = lr_train(X_extended, y)
    plot_2d_clf_problem(X, y, lambda x: lr_h(poly.fit_transform(x), w) >= 0.5)
    plt.show()
    print(cross_entropy_error(X_extended, y, w))
    print(X_extended[0])
    print(w.reshape(1, -1))

**Q:** Koji biste stupanj polinoma upotrijebili i zašto? Je li taj odabir povezan s odabirom regularizacijskog faktora $\alpha$? Zašto?